# Segmentace obrazu a měření rozměrů
Základní znalostí při zpracování obrazu je nejenom ideálně nasnímat obraz, ale hlavně ze získaného obrazu dostat použitelné informace. Extrakce objektů z obrazu (oddělení objektů od pozadí) se nazývá segmentace. Nejčastěji se využívá metod prahování, hranové detekce nebo kontur ([viz link](https://circuitdigest.com/tutorial/image-segmentation-using-opencv)).

Když už se podaří získat z obrazu objekty, je dále potřeba změřit jejich rozměry - a to nejen v pixelech, ale hlavně reálné rozměry v cm nebo dokonce v menších jednotkách. 

Na cvičení se bude využívat umělý obrázek s objekty, jejichž rozměry bude třeba automatizovaně získat.

![](images/index.jpg)

### Import knihoven a konfigurace

In [ ]:
%run ./library.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Seznam funkcí pro přehlednost:


Úkol 1
- [`load_image(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/acquisition/img_io.py#L11)
- [`plot_images(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/visualisation/visualisation.py#L11)
- [`crop(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L94)
- [`to_gray(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L4)
- [`segmentation_one_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/segmentation/segmentation.py#L50)
- [`segmentation_two_thresholds(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/segmentation/segmentation.py#L82)
- [`to_hsv(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L20)
- [`to_intensity(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/segmentation/segmentation.py#L13)
- [`cv2.inRange(...)`](https://docs.opencv.org/3.4.1/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981)
- [`cv2.minAreaRect(...)`](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9)
- [`cv2.boxPoints(...)`](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#gaf78d467e024b4d7936cf9397185d2f5c)

Úkol 2
- [`draw_rotated_text(...)`](./library.ipynb)
- [`draw_real_sizes(...)`](./library.ipynb)

---

### Úkol 1

Úkol je zaměřen na segmentace obrazů, získání kontur objektů a měření poměru rozměrů v cm a pixelech.

**1) Nasnímejte papír se vzory a uložte jej do složky data. Obrázek poté načtěte a zobrazte. Nakonec nadefinujte proměné s šířkou a výškou referečního objektu.**

Obrázek nastavte tak, aby byl ideálně záběr pouze na něj. *HINT: Je možné pomoci si oříznutím obrázku (crop).*

In [ ]:
img_patterns = ... ###
...(img_patterns)

In [ ]:
ref_width_real = ... ###
ref_height_real = ... ###

**2) Převeďte obrázek do šedotónu a segmentací získejte masku referenčního obdélníku.**


In [ ]:
img_patterns_gray = ... ###
...(img_patterns_gray)

Vyberte vhodnou funkci a experimentujte se sliderem (políčko s číselnými hodnotami je editovatelné).

In [ ]:
@interact(threshold_range=create_slider(min=0, max=255, description='Threshold range:',))
def _(threshold_range):
    mask = ...(img_patterns_gray, threshold_range[0], threshold_range[1]) ###
    plot_images(mask)

Optimální hodnoty zaneste do funkce:

In [ ]:
mask = ...(img_patterns_gray, ..., ...) ###

**3) BONUS: šedotónový obrázek je jen 2D numpy array, proveďte stejnou segmentaci pomocí numpy operací. (1b)**

Poznámka: for cyklus **není** numpy operace.

In [ ]:
def segment_gray_np(img, l_bound, u_bound):
    mask = ... ###
    return mask

In [ ]:
mask = segment_gray_np(img_patterns_gray, ..., ...)
plot_images(mask)

**4) Převeďte obrázek do do vhodné barevné soustavy, získejte masku referenčního obdélníku.**

*Hint: Pozor na rozsah hodnot v jednotlivých kanálech cv2.inRange, může se lišit v různých spektrech* 
![](https://i.stack.imgur.com/YOBFy.png)

In [ ]:
img_patterns_color = ...(img_patterns) ###

In [ ]:
@interact(h_range=create_slider(min=0, max=360, description='Hue:'),
          s_range=create_slider(min=0, max=255, description='Saturation:'),
          v_range=create_slider(min=0, max=255, description='Value:'))
def _(h_range, s_range, v_range):

    lower_bound = (to_intensity(...), ..., ...) ###
    upper_bound = (to_intensity(...), ..., ...) ###
    
    mask = ...(img_patterns_color,lower_bound,upper_bound) ###
    plot_images(mask, apply_mask(img_patterns, mask))

Optimální hodnoty zaneste do funkcí:

In [ ]:
lower_bound = ... ###
upper_bound = ... ###

mask = ...(img_patterns_color, lower_bound, upper_bound)

**5) Naleznete konturu která pokrývá nevětší plochu, otestujte aproximace kontury na obdélník.**

Výsledná maska by měla ideálně obsahovat pouze referenční obdelník, v obrazu se ale obecně může vyskytovat i šum. 
Nyní je nutné získat pozici referenčního obdelníku v obraze masky. 
Nejjednoduším způsobem je vyhledání kontur (obrysů) v obraze pomocí [cv2.findContours()](https://docs.opencv.org/3.3.1/d3/dc0/group__imgproc__shape.html#ga17ed9f5d79ae97bd4c7cf18403e1689a). Vyhledání kontur funguje jen na binárním černobílém obraze, proto jsme nejdříve museli využít cv2.inRange() k získání masky.

Konturu si lze představit jako křivku spojující několik bodů kolem obrysu souvislého objektu. Funkce cv2.findContours() příjímá navíc dva parametry, contour retrieval mode a contour approximation method. Těmito parametry se ve většině případů nemusíme zabývat. Zjednodušeně je tedy nastavíme `mode=cv2.RETR_LIST` a `method=cv2.CHAIN_APPROX_SIMPLE`. Pokud by vás přeci jen zajímaly, můžete si o nich přečíst [zde](https://docs.opencv.org/3.3.1/d4/d73/tutorial_py_contours_begin.html)

Vzhledem k šumu, který se na každém snímku vyskytuje, prakticky nikdy nenajdeme konturu pouze jednu. Je tedy nutné následně provést filtrování. V našem případě si vystačíme s výběrem kontury, která má největší plochu.

Posledním krokem je validace našeho postupu a vizualizace nalezené kontury pomocí funkce [cv2.drawContours()](https://docs.opencv.org/3.3.1/d6/d6e/group__imgproc__draw.html#ga746c0625f1781f1ffc9056259103edbc).
                                                                                

In [ ]:
contours, _  = ...(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) ###
print(f'Found {len(contours)} contours.')

# pick only the contour with the biggest area
contour_biggest = max(contours, key=cv2.contourArea)
print(f'Biggest contour area: {cv2.contourArea(contour_biggest)}, coordinates:\n {contour_biggest.reshape((-1, 2)).tolist()}')
contour_drawn = cv2.drawContours(img_patterns.copy(), [contour_biggest], -1, color=(255, 0, 0), thickness=5)
plot_images(contour_drawn)

Z výsledných souřadnic vidíme, že kontura netvoří přesný obdelník, proto ji musíme obdelníkem aproximovat. To lze pomocí funkce [cv2.minAreaRect](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9). Návratovou hodnotou této funkce je tuple - (střed obdélníku (x,y), (výška, šířka), úhel rotace obdélníku). Jedna otázka zní, co je výška a co je šířka? Musíte to vždycky kontrolovat okem.

Pomocná funkce [cv2.boxPoints()](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#gaf78d467e024b4d7936cf9397185d2f5c) převádí nalezený obdelník z formátu (střed obdélníku (x,y), (šířka, výška), úhel rotace obdélníku) na 4 rohové body obdélníku ve formátu (x, y). To se může hodit např. k vizualizaci (lze použít jako vstup pro cv2.drawContours()). Pozor, pořadí vrácených bodů není zaručeno.

In [ ]:
rect = ... ###
print(f'Rect tuple: {rect}')
print()
print(f'(cx, cy)={rect[0]}')
print(f'(height, width)={rect[1]}')
print(f'angle={rect[2]}')
print()
print(f'Rect points: {cv2.boxPoints(rect).tolist()}')

box_points_drawn = img_patterns.copy()
for p in np.int0(cv2.boxPoints(rect)):
    cv2.circle(box_points_drawn,(p[0],p[1]), 10, (0,0,255), -1)
plot_images(box_points_drawn)

**6) Nalezenou šířku referenčního obdelníku v pixelech můžeme konečně využít k získání poměru skutečné šířky obdélníku v reálných jednotkách a pixelové šířky obdélníku v obraze. Tento poměr budeme následně potřebovat k výpočtu rozměrů ostatních neznámých objektů.**

In [ ]:
ref_width_image, ref_height_image = ... ###
real_image_ratio = ... ### 
print(f'Ratio between real width and image width: {real_image_ratio}')

**7) Zkontrolujte si, že přepočtené hodnoty odpovídají reálné velikosti referenčního obdélníku.**

In [ ]:
print(f'real size: {(ref_width_real, ref_height_real)}')
print(f'recalculated size: {(ref_width_image*real_image_ratio, ref_height_image*real_image_ratio)}')

---

### Úkol 2

Úkol 1 máme za sebou a už víme poměr mezi rozměrem v cm a pixelech, tím pádem už můžeme naměřit rozměry ostatních objektů na obrázku s neznámými rozměry.

**1) Libovolným způsobem získejte masky objektů s neznámými skutečnými rozměry.**

In [ ]:
patterns_mask = ... ###
plot_images(patterns_mask)

**2) Nalezněte v nové masce snímku kontury stejně jako v případě referenčního objektu a následně proveďte filtrování kontur podle jejich obsahu. Prahovou hodnotu obsahu v pixelech (threshold) je nutno zvolit experimentálně.**

In [ ]:
contours, _  = cv2.findContours(patterns_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
print(f'Found {len(contours)} contours.')

# Filter out noise
threshold = ... ###
contours =  [c for c in contours if cv2.contourArea(c) > threshold]
print(f'After filtering, {len(contours)} contours remained.')

contour_drawn = cv2.drawContours(img_patterns.copy(), contours, -1, color=(255, 0, 0 ), thickness=10)
plot_images(contour_drawn)

# Sort contours by area. Just for better debugging.
contours.sort(key=cv2.contourArea, reverse=True)

**4) Nalezněte skutečné rozměry objektu**

Nyní potřebujeme pro jednotlivé kontury zjistit jejich skutečné rozměry, které chceme vizualizovat do výsledného obrázku.

Pro každou konturu tedy získáme jeji obdélníkovou aproximaci pomocí [cv2.minAreaRect](https://docs.opencv.org/3.1.0/d3/dc0/group__imgproc__shape.html#ga3d476a3417130ae5154aea421ca7ead9). Následně můžeme vypočítat skutečnou šířku a výšku objektu, díky předešlému vypočítanému poměru mezi skutečnou šířkou a pixelovou šířkou. Posledním krokem je volání funkce `draw_real_sizes()`, která se do vstupního obrázku pokusí vykreslit rozměry nalezeného objektu.

_Poznámka: pokud funkce `draw_real_sizes` vyhazuje chybu, změňte crop obrázku, jelikož funkce nemá dost místa, aby na okrajích vypsala text._

In [ ]:
# create a copy of original image
sizes_drawn = img_patterns.copy()

for c in contours:
    rect = ... ###
    shape_width, shape_height = ... ### 
    real_width = ...  ###
    real_height = ... ###
    print(rect[1], real_width, real_height)
    
    cv2.drawContours(sizes_drawn, [c], -1, color=(255, 0, 0 ), thickness=5)
    sizes_drawn = draw_real_sizes(
        ...,
        ...,
        ...,
        ..., 
        lbl_size_scale=.7,
        lbl_color=(0, 0, 255),
        lbl_thickness=1
    ) ###

plot_images(sizes_drawn)